
# Transformers y Mecanismos de Atención

Los **transformers** han revolucionado el procesamiento de lenguaje natural, visión por computador y series temporales, gracias a su capacidad de capturar relaciones a largo plazo y paralelizar el entrenamiento.

---

## Fundamentos

- **Atención**: mecanismo que pondera la importancia de cada entrada.
- **Self-Attention**: cada elemento presta atención a todos los demás.
- **Encoder-Decoder**: estructura usada en traducción y generación de texto.
- **Preentrenamiento**: BERT, GPT, T5, etc.

---

## Componentes clave:

- Embedding
- Positional Encoding
- Multi-Head Attention
- Feed-Forward Network
- Normalización y Residual



## Mecanismo de Atención (Escalar)

Dado una consulta (query) \( q \), claves (keys) \( k \) y valores (values) \( v \):

\[
\text{Attention}(q, K, V) = \text{softmax}\left(\frac{q K^T}{\sqrt{d_k}}\right) V
\]

Este mecanismo permite que el modelo "atienda" diferentes partes de la secuencia con distintas intensidades.


In [ ]:

import numpy as np

# Simulación simple
q = np.array([[1, 0]])    # Query
k = np.array([[1, 0], [0, 1]])  # Keys
v = np.array([[10, 0], [0, 10]])  # Values

# Atención escalar
score = q @ k.T
weights = np.exp(score) / np.sum(np.exp(score), axis=1, keepdims=True)
context = weights @ v

print("Contexto generado por atención:", context)



## Transformers con TensorFlow y Hugging Face

Utilizaremos un modelo preentrenado para clasificación de texto (`distilbert-base-uncased`).


In [ ]:

!pip install -q transformers datasets

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import load_dataset
import tensorflow as tf

# Cargar dataset y tokenizer
dataset = load_dataset("imdb", split="train[:2000]").train_test_split(test_size=0.2)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])

# Convertir a tf.data
train = tf.data.Dataset.from_tensor_slices((
    {'input_ids': dataset['train']['input_ids'], 'attention_mask': dataset['train']['attention_mask']},
    dataset['train']['label']
)).batch(16)

# Cargar modelo
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Entrenar
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train, epochs=1)



## Ejercicio Final

1. Prueba otro modelo como `bert-base-uncased` o `roberta-base`.
2. Evalúa sobre el conjunto de prueba.
3. Usa un modelo `AutoModelForQuestionAnswering` en un contexto simple.

¿Puedes adaptar la arquitectura para clasificación multiclase?
